In [1]:
import numpy as np
import cv2

S:\Users\sifat\anaconda3\envs\tf-gpu\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
S:\Users\sifat\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
S:\Users\sifat\anaconda3\envs\tf-gpu\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
corner_track_params = dict(maxCorners = 10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [3]:
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
## Lucas Kanade method for optical flow tracking a few points
## and then drawing lines from their previous position to their 
## next position

cap = cv2.VideoCapture(0)
ret,prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# PTS TO TRACK
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)
mask = np.zeros_like(prev_frame)

while True:
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    nextPts, status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i ,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2) ## this is how the pts are gonna
                                       ## accept in Lucas Kanade method

cv2.destroyAllWindows()
cap.release()

In [5]:
# a = np.array([[1,2,3,4],[2,1,1,1]])
# b = np.array([[2,1,0,4],[2,1,3,0]])
# a.shape

In [6]:
# for i , (x,y) in enumerate(zip(a,b)):
#     a,b,c,d = x
#     print(a,b,c,d)

In [11]:
## Dense Optical Flow
## HIghlighting image when it starts to move to any direction

cap = cv2.VideoCapture(0)

ret,fram1=cap.read()
prvsImg = cv2.cvtColor(fram1,cv2.COLOR_BGR2GRAY)
hsv_mask = np.zeros_like(fram1)
hsv_mask[:,:,1] = 255

while True:
    ret,frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
#     print(flow.shape)
    mag,ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
#     print(mag,ang)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k==27:
        break
    prvsImg = nextImg
    
cap.release()
cv2.destroyAllWindows()
    